In [ ]:
# Downloaded from Uniprot FTP server and unzipped

trembl_file_path = "~/uniprot_2022_04/uniprot_trembl.fasta"
id_selected_file_path = "~/uniprot_2022_04/idmapping_selected.tab"

In [ ]:
with open(trembl_file_path) as trembl_file:
    for line in trembl_file:
        print(line)
        break